In [ ]:
import torch
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output
from bindsnet.models import DiehlAndCook2015
from bindsnet.network.monitors import Monitor
from bindsnet.encoding import poisson
from encoder import *
import time as T
plt.rcParams["figure.figsize"] = (20, 20)
lfname = 'logs/' + str(int(T.time())) + '_normal.txt'

In [ ]:
time = 500
network = DiehlAndCook2015(100*100*2, dt=1.0, norm=2000, inh=3)
exc_monitor = Monitor(network.layers['Ae'], ['v', 's'], time=time)
network.add_monitor(exc_monitor, name='exc')

In [ ]:
track1 = pickle.load(open('encoding/test_normal.p', 'rb'))

In [ ]:
def log(msg):
    print(msg, end='')
    with open(lfname, 'a') as f:
        f.write(msg)

In [ ]:
def detect(spikes):
    cols = torch.sum(spikes, dim=1)
    for i, v in enumerate(cols):
        if v != 0:
            log(f'Neuron: {i} Spikes: {int(v)}\n')

In [ ]:
for i in range(1, len(track1)):
    try:
        orig = torch.cat((track1[i-1], track1[i]), dim=1)
        pt = orig.view(-1)
        pt = poisson(pt, time)
        
        inpts = {'X': pt}
        network.run(inpts=inpts, time=time)
        spikes = exc_monitor.get('s')
        
        log(f'Iteration {i}\n')
        detect(spikes)
        
        fig = plt.figure(figsize=(20, 20))
        plt.subplot(2, 2, 1)
        plt.imshow(spikes, cmap='binary')
        plt.subplot(2, 2, 2)
        plt.imshow(orig, cmap='gist_gray')
        plt.show()
        
        clear_output(wait=True)
    except KeyboardInterrupt:
        break

In [ ]:
network.save('trained.net')